In [30]:
import os
import re
import pandas as pd
import numpy as np
from ast import literal_eval

In [31]:
# Read in bounding box files
box_files = os.listdir("../bounding_boxes/word_coordinates_split_6_4")
box_files.sort()

# Read in AST files
annotated_files = os.listdir("annotations_6_4")
annotated_files.sort()

# Sanity check to test whether the files line up by index after sorting
for i, file in enumerate(box_files):
    box_name = re.sub("_boxes.csv", "", file)
    ann_name = re.sub("_annotated.csv", "", annotated_files[i])
    if box_name != ann_name:
        print("mismatch: ", box_name, ann_name)

In [36]:
#####################################
### FUNCTIONS USED TO MERGE FILES ###
#####################################


# function that takes in the original dataframe and the dataframe 
# only containing comments/strings. This merges the multi-line comments/strings,
# puts the merged string in the word column, and the original word in the word column
# returns the merged dataframe
def merge_words(og_df, new_df):
    # stops pandas from interpreting 'NaN' as a float
    new_df['word'] = new_df['word'].astype(str) 

    # actual joining based on the occurence number
    mid_df = new_df.groupby('occurrence')['word'].apply(' '.join).reset_index()
    mid_df.columns = ['occurrence', 'word']
    mid_df = pd.merge(og_df, mid_df, on='occurrence', how="left")
    
    # moving merged string to word column
    mid_df['word'] = mid_df['word_y'].where(mid_df['word_y'].notna(), mid_df['word_x'])
    mid_df = mid_df.drop(columns = ['word_x', 'word_y']) # dropping extra columns that get created
    
    return mid_df


# Using this function to merge the bounding boxes and AST files
def merge_files(bb, ast):
    # separating because merge based on word column for strings/comments 
    # and word + occurrence column for the rest
    strings = bb[bb['occurrence'] >= 100]
    therest = bb[bb['occurrence'] < 100]
    
    # two separate merges
    strdf = pd.merge(strings, ast, on='word', how='left')
    strdf = strdf.rename(columns = {'occurrence_x': 'occurrence'})
    # FIXME need to fix column names so occurrence doesn't get lost
    # print(strdf['occurrence_x'])
    # print(strdf['occurrence_y'])
    restdf = pd.merge(therest, ast, on=['word', 'occurrence'], how='left')

    # merging the merges, sorting to original order, and reordering the columns
    merged = pd.concat([strdf, restdf])
    merged = merged.sort_values(by=['y', 'x'])
    merged = merged[['word', 'occurrence', 'original', 'tree_parts', 'x', 'y', 'width', 'height',
                    'tobii_x', 'tobii_y', 'tobii_width', 'tobii_height']]
    
    return merged

# For cases where a comment or string isn't matched to something from the ast file,
# this function fills in that row with the contents above or below, and logs whenever that happens
def fillna_and_log(df, filename, method=None):
    df_before = df.copy()  # Copy of the DataFrame before filling
    df_after = df.copy()
    df_after['tree_parts'] = df['tree_parts'].fillna(method=method)  # DataFrame after filling tree parts column

    before = df_before['tree_parts']
    after = df_after['tree_parts']

    # Find the indices where a NaN was filled
    filled_indices = np.where(before.isna() & ~after.isna())[0]

    # For each index where a NaN was filled
    for idx in filled_indices:
        parts = literal_eval(df_after.iloc[idx, 3]) # interpreting tree parts as a list, and not a string
        kind = df_after.iloc[idx, 1] # whether it's a string or comment, or neither
        parts.pop() # removing last element. Not a perfect solution, but an ok one
        
        if kind >= 200: # if it's a comment
            parts.append('comment')
        elif kind >= 100 and kind < 200: # if it's a string
            parts.append('literal')
        else: # could be a random thing like ) or };
            parts = []
            #print(f"Tree parts missing for {df.iloc[idx, 0]}-------------")
        df_after.iat[idx, 3] = parts
        #print(f"Filled in tree parts for {df.iloc[idx, 0]} at index {idx} with value {parts}")
    return df_after


In [37]:
# Actual work of merging happens here 

for i, file in enumerate(box_files):
    print(file)
    boxes = pd.read_csv(f"../bounding_boxes/word_coordinates_split_6_4/{file}")
    ast = pd.read_csv(f"annotations_6_4/{annotated_files[i]}")
    name = re.sub("_boxes", "", file)
    # moving strings and comments to 'original' column
    boxes['original'] = boxes['word'].where(boxes['occurrence'] >= 100) #, boxes['original'])

    # filtering to get just comments and just strings
    comments = boxes[(boxes['occurrence'] >= 200)]
    strings = boxes[(boxes['occurrence'] >= 100) & (boxes['occurrence'] < 200)]
    
    merged_comments = merge_words(boxes, comments)
    merged_strings = merge_words(merged_comments, strings)

    # rearranging columns
    merged_strings = merged_strings[['word', 'occurrence', 'original', 'x', 'y', 'width', 
                                     'height', 'tobii_x', 'tobii_y', 'tobii_width', 'tobii_height']]

    # actual merging
    wholedf = merge_files(merged_strings, ast) 
    wholedf = fillna_and_log(wholedf, name, 'ffill') # for strings/comments that don't equal strings exactly
    wholedf.to_csv(f'fully_annotated/{name}', index=False)
    

BFSdist_boxes.csv
abstractMatrix3D_boxes.csv
actionLoadNotes_boxes.csv
actionPerformed_boxes.csv
Tree parts missing for ]-------------
Tree parts missing for ))-------------
Tree parts missing for ]-------------
addDbArgs_boxes.csv


/tmp/ipykernel_454842/1762274316.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['word'] = new_df['word'].astype(str)
/tmp/ipykernel_454842/1762274316.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['word'] = new_df['word'].astype(str)
/tmp/ipykernel_454842/1762274316.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

addDigit_boxes.csv
addErrors_boxes.csv
addLimit_boxes.csv
addPKColumn_boxes.csv
addRelativeHeaders_boxes.csv
addRotation_boxes.csv
addToTechnicalComment_boxes.csv


/tmp/ipykernel_454842/1762274316.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['word'] = new_df['word'].astype(str)
/tmp/ipykernel_454842/1762274316.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['word'] = new_df['word'].astype(str)


addUIDeployment_boxes.csv
add_boxes.csv
appendDeclarations_boxes.csv
Tree parts missing for )-------------
asMap_boxes.csv
atomToExpression_boxes.csv
capitalizeString_boxes.csv
checkDescendantM_boxes.csv
checkSetLayout_boxes.csv


...

Tree parts missing for )-------------
setPhoto_boxes.csv
setSecurityMode_boxes.csv
setSelectedScript_boxes.csv
setTopAttributes_boxes.csv
setUpMrj_boxes.csv
Tree parts missing for ))-------------
setUrls_boxes.csv
showLatestPlan_boxes.csv
show_boxes.csv
split_boxes.csv
store_boxes.csv
swapItems_boxes.csv
tellAgain_boxes.csv


/tmp/ipykernel_454842/1762274316.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['word'] = new_df['word'].astype(str)
/tmp/ipykernel_454842/1762274316.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['word'] = new_df['word'].astype(str)
/tmp/ipykernel_454842/1762274316.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

testAddCountryWithSequenceGenerator_boxes.csv
testCookieGreaterThan_boxes.csv
testGetClassification_boxes.csv
testGetCodigo_boxes.csv
testGetEmail_boxes.csv
testGetEvtIDs_boxes.csv
testInvoke_boxes.csv
testLogin_boxes.csv
testMoveRFWithNullContainer_boxes.csv
testNegativeParseCases_boxes.csv
testOccThm_boxes.csv
testOneTwoThreeCreateCycle_boxes.csv
testSetChkEvtID_boxes.csv
testSetEvtID_boxes.csv
testSetExample_boxes.csv
testSetWelcomeMsg_boxes.csv
testValidateSeparationCount_boxes.csv
test_10_bug2689872_boxes.csv
toString_boxes.csv
updateGain_boxes.csv
updateSchema_boxes.csv
visitRetStmt_boxes.csv


/tmp/ipykernel_454842/1762274316.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['word'] = new_df['word'].astype(str)
/tmp/ipykernel_454842/1762274316.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['word'] = new_df['word'].astype(str)
/tmp/ipykernel_454842/1762274316.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

wndPreferences_boxes.csv
